# SQL Warmups: Sequencing QC Data Analysis

This notebook contains 25 SQL warmup exercises using a PostgreSQL database populated with sequencing QC data from 10 clinical patient samples.

## 1. Setup and Database Connection

In [ ]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import random
from sqlalchemy import create_engine

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Database connection parameters
DB_USER = 'shanebrubaker'  # Change to your username
DB_PASSWORD = ''  # Add password if needed (usually empty for local)
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sequencing_qc'

# Create SQLAlchemy engine (this eliminates pandas warning)
engine = create_engine(f'postgresql://{DB_USER}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# Also keep psycopg2 connection for database setup operations
def get_psycopg2_connection():
    return psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )

# Test connection
try:
    with engine.connect() as conn:
        print("Successfully connected to PostgreSQL database using SQLAlchemy!")
except Exception as e:
    print(f"Error connecting to database: {e}")

## 2. Database Schema Creation and Data Population

In [ ]:
# Drop existing tables if they exist and create schema
conn = get_psycopg2_connection()
cur = conn.cursor()

# Drop tables
cur.execute("""
    DROP TABLE IF EXISTS qc_metrics CASCADE;
    DROP TABLE IF EXISTS sequencing_runs CASCADE;
    DROP TABLE IF EXISTS samples CASCADE;
    DROP TABLE IF EXISTS patients CASCADE;
""")

# Create patients table
cur.execute("""
    CREATE TABLE patients (
        patient_id VARCHAR(20) PRIMARY KEY,
        age INTEGER,
        gender VARCHAR(10),
        diagnosis VARCHAR(100),
        enrollment_date DATE
    )
""")

# Create samples table
cur.execute("""
    CREATE TABLE samples (
        sample_id VARCHAR(20) PRIMARY KEY,
        patient_id VARCHAR(20) REFERENCES patients(patient_id),
        sample_type VARCHAR(50),
        collection_date DATE,
        tissue_source VARCHAR(50),
        dna_concentration DECIMAL(10, 2)
    )
""")

# Create sequencing_runs table
cur.execute("""
    CREATE TABLE sequencing_runs (
        run_id VARCHAR(20) PRIMARY KEY,
        sample_id VARCHAR(20) REFERENCES samples(sample_id),
        sequencer VARCHAR(50),
        run_date DATE,
        platform VARCHAR(50),
        library_prep VARCHAR(50)
    )
""")

# Create qc_metrics table
cur.execute("""
    CREATE TABLE qc_metrics (
        metric_id SERIAL PRIMARY KEY,
        run_id VARCHAR(20) REFERENCES sequencing_runs(run_id),
        total_reads BIGINT,
        mapped_reads BIGINT,
        duplicate_reads BIGINT,
        mean_coverage DECIMAL(10, 2),
        mean_quality_score DECIMAL(5, 2),
        gc_content DECIMAL(5, 2),
        insert_size_mean DECIMAL(8, 2),
        error_rate DECIMAL(5, 4),
        pass_filter BOOLEAN
    )
""")

conn.commit()
print("Tables created successfully!")

In [ ]:
# Populate with faux sequencing QC data
random.seed(42)
np.random.seed(42)

# Generate patient data
diagnoses = ['Breast Cancer', 'Lung Cancer', 'Colorectal Cancer', 'Leukemia', 'Lymphoma']
base_date = datetime(2024, 1, 1)

patients_data = []
for i in range(1, 11):
    patient_id = f'PT{i:04d}'
    age = random.randint(25, 85)
    gender = random.choice(['Male', 'Female'])
    diagnosis = random.choice(diagnoses)
    enrollment_date = base_date + timedelta(days=random.randint(0, 200))
    patients_data.append((patient_id, age, gender, diagnosis, enrollment_date))

cur.executemany("""
    INSERT INTO patients (patient_id, age, gender, diagnosis, enrollment_date)
    VALUES (%s, %s, %s, %s, %s)
""", patients_data)

# Generate sample data (1-2 samples per patient)
sample_types = ['Tumor', 'Normal', 'Blood']
tissue_sources = ['Breast', 'Lung', 'Colon', 'Bone Marrow', 'Lymph Node', 'Blood']

samples_data = []
sample_counter = 1
for patient_id, _, _, _, _ in patients_data:
    num_samples = random.randint(1, 2)
    for _ in range(num_samples):
        sample_id = f'S{sample_counter:04d}'
        sample_type = random.choice(sample_types)
        collection_date = base_date + timedelta(days=random.randint(0, 250))
        tissue_source = random.choice(tissue_sources)
        dna_concentration = round(random.uniform(10.0, 250.0), 2)
        samples_data.append((sample_id, patient_id, sample_type, collection_date, tissue_source, dna_concentration))
        sample_counter += 1

cur.executemany("""
    INSERT INTO samples (sample_id, patient_id, sample_type, collection_date, tissue_source, dna_concentration)
    VALUES (%s, %s, %s, %s, %s, %s)
""", samples_data)

# Generate sequencing runs (1 run per sample)
sequencers = ['NextSeq550', 'NovaSeq6000', 'MiSeq']
platforms = ['Illumina']
library_preps = ['TruSeq DNA', 'Nextera XT', 'TruSeq RNA']

runs_data = []
for idx, (sample_id, _, _, _, _, _) in enumerate(samples_data, 1):
    run_id = f'RUN{idx:04d}'
    sequencer = random.choice(sequencers)
    run_date = base_date + timedelta(days=random.randint(50, 300))
    platform = random.choice(platforms)
    library_prep = random.choice(library_preps)
    runs_data.append((run_id, sample_id, sequencer, run_date, platform, library_prep))

cur.executemany("""
    INSERT INTO sequencing_runs (run_id, sample_id, sequencer, run_date, platform, library_prep)
    VALUES (%s, %s, %s, %s, %s, %s)
""", runs_data)

# Generate QC metrics
qc_data = []
for run_id, _, _, _, _, _ in runs_data:
    total_reads = random.randint(50_000_000, 200_000_000)
    mapped_reads = int(total_reads * random.uniform(0.85, 0.98))
    duplicate_reads = int(total_reads * random.uniform(0.05, 0.25))
    mean_coverage = round(random.uniform(30.0, 150.0), 2)
    mean_quality_score = round(random.uniform(30.0, 38.0), 2)
    gc_content = round(random.uniform(38.0, 48.0), 2)
    insert_size_mean = round(random.uniform(300.0, 500.0), 2)
    error_rate = round(random.uniform(0.001, 0.015), 4)
    pass_filter = mean_quality_score > 30 and mean_coverage > 30
    
    qc_data.append((run_id, total_reads, mapped_reads, duplicate_reads, mean_coverage, 
                   mean_quality_score, gc_content, insert_size_mean, error_rate, pass_filter))

cur.executemany("""
    INSERT INTO qc_metrics (run_id, total_reads, mapped_reads, duplicate_reads, mean_coverage,
                           mean_quality_score, gc_content, insert_size_mean, error_rate, pass_filter)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", qc_data)

conn.commit()
cur.close()
conn.close()

print(f"Data populated successfully!")
print(f"- {len(patients_data)} patients")
print(f"- {len(samples_data)} samples")
print(f"- {len(runs_data)} sequencing runs")
print(f"- {len(qc_data)} QC metric records")

## 3. SQL Warmup Queries

Below are 25 SQL warmup queries of increasing complexity.

### Query 1: Select all patients

In [ ]:
query = "SELECT * FROM patients;"
df = pd.read_sql(query, engine)
display(df)

### Query 2: Count total number of samples

In [ ]:
query = "SELECT COUNT(*) as total_samples FROM samples;"
df = pd.read_sql(query, engine)
display(df)

### Query 3: Find all female patients

In [ ]:
query = "SELECT * FROM patients WHERE gender = 'Female';"
df = pd.read_sql(query, engine)
display(df)

### Query 4: Get patients older than 60

In [ ]:
query = "SELECT patient_id, age, gender, diagnosis FROM patients WHERE age > 60 ORDER BY age DESC;"
df = pd.read_sql(query, engine)
display(df)

### Query 5: List all unique sample types

In [ ]:
query = "SELECT DISTINCT sample_type FROM samples;"
df = pd.read_sql(query, engine)
display(df)

### Query 6: Get samples with DNA concentration > 100 ng/uL

In [ ]:
query = "SELECT sample_id, sample_type, dna_concentration FROM samples WHERE dna_concentration > 100 ORDER BY dna_concentration DESC;"
df = pd.read_sql(query, engine)
display(df)

### Query 7: Count sequencing runs by sequencer type

In [ ]:
query = "SELECT sequencer, COUNT(*) as run_count FROM sequencing_runs GROUP BY sequencer ORDER BY run_count DESC;"
df = pd.read_sql(query, engine)
display(df)

### Query 8: Calculate average QC metrics

In [ ]:
query = """
    SELECT 
        ROUND(AVG(mean_coverage), 2) as avg_coverage,
        ROUND(AVG(mean_quality_score), 2) as avg_quality,
        ROUND(AVG(gc_content), 2) as avg_gc_content
    FROM qc_metrics;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 9: Find runs that passed QC filter

In [ ]:
query = """
    SELECT sr.run_id, sr.sequencer, qm.mean_coverage, qm.mean_quality_score
    FROM sequencing_runs sr
    JOIN qc_metrics qm ON sr.run_id = qm.run_id
    WHERE qm.pass_filter = TRUE;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 10: Get sample information with patient details

In [ ]:
query = """
    SELECT s.sample_id, s.sample_type, p.patient_id, p.age, p.diagnosis
    FROM samples s
    JOIN patients p ON s.patient_id = p.patient_id
    ORDER BY p.age DESC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 11: Count patients by diagnosis

In [ ]:
query = "SELECT diagnosis, COUNT(*) as patient_count FROM patients GROUP BY diagnosis ORDER BY patient_count DESC;"
df = pd.read_sql(query, engine)
display(df)

### Query 12: Find runs with coverage below 50X

In [ ]:
query = """
    SELECT run_id, mean_coverage, mean_quality_score
    FROM qc_metrics
    WHERE mean_coverage < 50
    ORDER BY mean_coverage ASC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 13: Calculate mapping rate for each run

In [ ]:
query = """
    SELECT 
        run_id,
        total_reads,
        mapped_reads,
        ROUND((mapped_reads::DECIMAL / total_reads * 100), 2) as mapping_rate_pct
    FROM qc_metrics
    ORDER BY mapping_rate_pct DESC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 14: Get the most recent sequencing run for each sample

In [ ]:
query = """
    SELECT sample_id, run_id, sequencer, run_date
    FROM sequencing_runs
    WHERE run_date IN (
        SELECT MAX(run_date)
        FROM sequencing_runs
        GROUP BY sample_id
    )
    ORDER BY run_date DESC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 15: Find samples from Breast Cancer patients

In [ ]:
query = """
    SELECT s.sample_id, s.sample_type, p.patient_id, p.diagnosis
    FROM samples s
    JOIN patients p ON s.patient_id = p.patient_id
    WHERE p.diagnosis = 'Breast Cancer';
"""
df = pd.read_sql(query, engine)
display(df)

### Query 16: Calculate duplicate rate for each run

In [ ]:
query = """
    SELECT 
        run_id,
        duplicate_reads,
        total_reads,
        ROUND((duplicate_reads::DECIMAL / total_reads * 100), 2) as duplicate_rate_pct
    FROM qc_metrics
    ORDER BY duplicate_rate_pct DESC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 17: Get complete information for all runs (JOIN all tables)

In [ ]:
query = """
    SELECT 
        p.patient_id,
        p.diagnosis,
        s.sample_id,
        s.sample_type,
        sr.run_id,
        sr.sequencer,
        qm.mean_coverage,
        qm.mean_quality_score,
        qm.pass_filter
    FROM patients p
    JOIN samples s ON p.patient_id = s.patient_id
    JOIN sequencing_runs sr ON s.sample_id = sr.sample_id
    JOIN qc_metrics qm ON sr.run_id = qm.run_id
    ORDER BY p.patient_id;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 18: Count samples by tissue source

In [ ]:
query = "SELECT tissue_source, COUNT(*) as sample_count FROM samples GROUP BY tissue_source ORDER BY sample_count DESC;"
df = pd.read_sql(query, engine)
display(df)

### Query 19: Find runs with quality score above 35

In [ ]:
query = """
    SELECT sr.run_id, sr.sequencer, qm.mean_quality_score, qm.mean_coverage
    FROM sequencing_runs sr
    JOIN qc_metrics qm ON sr.run_id = qm.run_id
    WHERE qm.mean_quality_score > 35
    ORDER BY qm.mean_quality_score DESC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 20: Calculate average coverage by sequencer type

In [ ]:
query = """
    SELECT 
        sr.sequencer,
        ROUND(AVG(qm.mean_coverage), 2) as avg_coverage,
        COUNT(*) as run_count
    FROM sequencing_runs sr
    JOIN qc_metrics qm ON sr.run_id = qm.run_id
    GROUP BY sr.sequencer
    ORDER BY avg_coverage DESC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 21: Use CASE to categorize coverage levels

In [ ]:
query = """
    SELECT 
        run_id,
        mean_coverage,
        CASE 
            WHEN mean_coverage < 30 THEN 'Low'
            WHEN mean_coverage BETWEEN 30 AND 100 THEN 'Medium'
            ELSE 'High'
        END as coverage_category
    FROM qc_metrics
    ORDER BY mean_coverage DESC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 22: Find patients with multiple samples

In [ ]:
query = """
    SELECT p.patient_id, p.diagnosis, COUNT(s.sample_id) as sample_count
    FROM patients p
    JOIN samples s ON p.patient_id = s.patient_id
    GROUP BY p.patient_id, p.diagnosis
    HAVING COUNT(s.sample_id) > 1
    ORDER BY sample_count DESC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 23: Calculate min, max, and average total reads

In [ ]:
query = """
    SELECT 
        MIN(total_reads) as min_reads,
        MAX(total_reads) as max_reads,
        ROUND(AVG(total_reads)) as avg_reads
    FROM qc_metrics;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 24: Find runs with above-average coverage using subquery

In [ ]:
query = """
    SELECT run_id, mean_coverage
    FROM qc_metrics
    WHERE mean_coverage > (SELECT AVG(mean_coverage) FROM qc_metrics)
    ORDER BY mean_coverage DESC;
"""
df = pd.read_sql(query, engine)
display(df)

### Query 25: Complex aggregation - QC pass rate by diagnosis

In [ ]:
query = """
    SELECT 
        p.diagnosis,
        COUNT(*) as total_runs,
        SUM(CASE WHEN qm.pass_filter = TRUE THEN 1 ELSE 0 END) as passed_runs,
        ROUND(
            SUM(CASE WHEN qm.pass_filter = TRUE THEN 1 ELSE 0 END)::DECIMAL / COUNT(*) * 100, 
            2
        ) as pass_rate_pct
    FROM patients p
    JOIN samples s ON p.patient_id = s.patient_id
    JOIN sequencing_runs sr ON s.sample_id = sr.sample_id
    JOIN qc_metrics qm ON sr.run_id = qm.run_id
    GROUP BY p.diagnosis
    ORDER BY pass_rate_pct DESC;
"""
df = pd.read_sql(query, engine)
display(df)

## 4. Data Visualizations

### Visualization 1: Distribution of Mean Coverage Across Runs

In [ ]:
query = "SELECT mean_coverage FROM qc_metrics;"
df = pd.read_sql(query, engine)

plt.figure(figsize=(10, 6))
plt.hist(df['mean_coverage'], bins=15, color='steelblue', edgecolor='black', alpha=0.7)
plt.axvline(df['mean_coverage'].mean(), color='red', linestyle='--', linewidth=2, label=f"Mean: {df['mean_coverage'].mean():.2f}X")
plt.xlabel('Mean Coverage (X)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Mean Coverage Across Sequencing Runs', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### Visualization 2: Quality Scores by Sequencer Type

In [ ]:
query = """
    SELECT sr.sequencer, qm.mean_quality_score
    FROM sequencing_runs sr
    JOIN qc_metrics qm ON sr.run_id = qm.run_id;
"""
df = pd.read_sql(query, engine)

plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='sequencer', y='mean_quality_score', hue='sequencer', palette='Set2', legend=False)
plt.xlabel('Sequencer', fontsize=12)
plt.ylabel('Mean Quality Score', fontsize=12)
plt.title('Quality Score Distribution by Sequencer Type', fontsize=14, fontweight='bold')
plt.axhline(y=30, color='red', linestyle='--', linewidth=1, label='Q30 Threshold')
plt.legend()
plt.tight_layout()
plt.show()

### Visualization 3: Mapping Rate vs Duplicate Rate Scatter Plot

In [ ]:
query = """
    SELECT 
        ROUND((mapped_reads::DECIMAL / total_reads * 100), 2) as mapping_rate,
        ROUND((duplicate_reads::DECIMAL / total_reads * 100), 2) as duplicate_rate,
        pass_filter
    FROM qc_metrics;
"""
df = pd.read_sql(query, engine)

plt.figure(figsize=(10, 6))
colors = ['green' if pf else 'red' for pf in df['pass_filter']]
plt.scatter(df['mapping_rate'], df['duplicate_rate'], c=colors, alpha=0.6, s=100, edgecolors='black')
plt.xlabel('Mapping Rate (%)', fontsize=12)
plt.ylabel('Duplicate Rate (%)', fontsize=12)
plt.title('Mapping Rate vs Duplicate Rate', fontsize=14, fontweight='bold')
plt.legend(['Failed QC', 'Passed QC'], loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Visualization 4: Sample Count by Diagnosis

In [ ]:
query = """
    SELECT p.diagnosis, COUNT(s.sample_id) as sample_count
    FROM patients p
    JOIN samples s ON p.patient_id = s.patient_id
    GROUP BY p.diagnosis
    ORDER BY sample_count DESC;
"""
df = pd.read_sql(query, engine)

plt.figure(figsize=(10, 6))
plt.barh(df['diagnosis'], df['sample_count'], color='coral', edgecolor='black')
plt.xlabel('Number of Samples', fontsize=12)
plt.ylabel('Diagnosis', fontsize=12)
plt.title('Sample Count by Diagnosis', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### Visualization 5: Coverage vs Quality Score Relationship

In [ ]:
query = "SELECT mean_coverage, mean_quality_score, pass_filter FROM qc_metrics;"
df = pd.read_sql(query, engine)

plt.figure(figsize=(10, 6))
passed = df[df['pass_filter'] == True]
failed = df[df['pass_filter'] == False]

plt.scatter(passed['mean_coverage'], passed['mean_quality_score'], 
           color='green', alpha=0.6, s=100, label='Passed QC', edgecolors='black')
plt.scatter(failed['mean_coverage'], failed['mean_quality_score'], 
           color='red', alpha=0.6, s=100, label='Failed QC', edgecolors='black')

plt.xlabel('Mean Coverage (X)', fontsize=12)
plt.ylabel('Mean Quality Score', fontsize=12)
plt.title('Coverage vs Quality Score Relationship', fontsize=14, fontweight='bold')
plt.axhline(y=30, color='gray', linestyle='--', linewidth=1, alpha=0.5)
plt.axvline(x=30, color='gray', linestyle='--', linewidth=1, alpha=0.5)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Visualization 6: Total Reads by Sequencer (Bar Chart)

In [ ]:
query = """
    SELECT sr.sequencer, AVG(qm.total_reads) as avg_reads
    FROM sequencing_runs sr
    JOIN qc_metrics qm ON sr.run_id = qm.run_id
    GROUP BY sr.sequencer
    ORDER BY avg_reads DESC;
"""
df = pd.read_sql(query, engine)

plt.figure(figsize=(10, 6))
bars = plt.bar(df['sequencer'], df['avg_reads']/1e6, color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black')
plt.xlabel('Sequencer', fontsize=12)
plt.ylabel('Average Total Reads (Millions)', fontsize=12)
plt.title('Average Total Reads by Sequencer Type', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}M',
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

### Visualization 7: QC Pass Rate Heatmap by Sequencer and Library Prep

In [ ]:
query = """
    SELECT 
        sr.sequencer,
        sr.library_prep,
        ROUND(
            SUM(CASE WHEN qm.pass_filter = TRUE THEN 1 ELSE 0 END)::DECIMAL / COUNT(*) * 100, 
            1
        ) as pass_rate
    FROM sequencing_runs sr
    JOIN qc_metrics qm ON sr.run_id = qm.run_id
    GROUP BY sr.sequencer, sr.library_prep;
"""
df = pd.read_sql(query, engine)

# Pivot for heatmap
pivot_df = df.pivot(index='library_prep', columns='sequencer', values='pass_rate')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_df, annot=True, fmt='.1f', cmap='RdYlGn', center=75, 
            cbar_kws={'label': 'QC Pass Rate (%)'}, linewidths=1, linecolor='black')
plt.title('QC Pass Rate by Sequencer and Library Prep', fontsize=14, fontweight='bold')
plt.xlabel('Sequencer', fontsize=12)
plt.ylabel('Library Prep', fontsize=12)
plt.tight_layout()
plt.show()

## 5. Summary Statistics

In [ ]:
query = """
    SELECT 
        COUNT(DISTINCT p.patient_id) as total_patients,
        COUNT(DISTINCT s.sample_id) as total_samples,
        COUNT(DISTINCT sr.run_id) as total_runs,
        SUM(CASE WHEN qm.pass_filter = TRUE THEN 1 ELSE 0 END) as passed_qc,
        ROUND(AVG(qm.mean_coverage), 2) as avg_coverage,
        ROUND(AVG(qm.mean_quality_score), 2) as avg_quality
    FROM patients p
    JOIN samples s ON p.patient_id = s.patient_id
    JOIN sequencing_runs sr ON s.sample_id = sr.sample_id
    JOIN qc_metrics qm ON sr.run_id = qm.run_id;
"""
df = pd.read_sql(query, engine)
print("\n=== Overall Summary Statistics ===")
display(df)

In [ ]:
## End of Notebook ##